In [36]:
import yfinance as yf
import numpy as np
import pandas as pd
from ta.volatility import BollingerBands
pd.options.mode.chained_assignment = None  # default='warn'

In [37]:
def prepare_data(tickers, start, end, window_dev_1, window_dev_2):
    
    data = yf.download(tickers, start=start, end=end)

    #Store only the close prices
    df = data['Close']

    # Calculate Double Bollinger Bands variables

    for stock in tickers:
        # Calculate standard Bollinger Bands (using window_dev_1 standard deviation)
        bb_1sd = BollingerBands(close=df[stock], window=20, window_dev=window_dev_1)
        df[f'{stock}_ub_1'] = bb_1sd.bollinger_hband()
        df[f'{stock}_lb_1'] = bb_1sd.bollinger_lband()
        
        # Calculate standard Bollinger Bands (using window_dev_2 standard deviation)
        bb_2sd = BollingerBands(close=df[stock], window=20, window_dev=window_dev_2)
        df[f'{stock}_ub_2'] = bb_2sd.bollinger_hband()
        df[f'{stock}_lb_2'] = bb_2sd.bollinger_lband()
    df = df.dropna()
    return df

In [43]:
def Double_Bollinger_Bands(data, stock:str, start, end, initial_capital):
    buy_signal = False
    sell_signal = False

    current_capital = initial_capital
    shares_held = 0
    position_value = 0

    columns = data.filter(like=stock).columns.values
    stock_data = data[columns]
    # filter the backtest date
    bt_data = stock_data.loc[start:end] 
    

    
    for index in range(len(bt_data)-1):
        row = bt_data.iloc[index]
        next_row = bt_data.iloc[index+1]
        
        
        # Check buy signals (when price is between two upper bands)
        if row[stock] > row[f'{stock}_ub_1'] and row[stock] < row[f'{stock}_ub_2']:
            buy_signal = True
            bt_data.loc[bt_data.index[index],'Signal'] ='BUY'
        
        # Check  sell signals (when price is between two lower bands)
        elif row[stock] < row[f'{stock}_lb_1'] and row[stock] > row[f'{stock}_lb_2']:
            sell_signal = True
            bt_data.loc[bt_data.index[index],'Signal'] ='SELL'


        if buy_signal and current_capital>= next_row[stock]:
            shares_bought = int(current_capital/next_row[stock])
            cost = shares_bought * next_row[stock]
            current_capital -= cost  
            shares_held += shares_bought  
            buy_signal = False  # reset buy signal
        
        if sell_signal and shares_held>0:
            current_capital += shares_bought*next_row[stock]
            shares_held = 0
            sell_signal = False
        
        # Update everyday's asset value
        next_date = bt_data.index[index+1]
        bt_data.loc[next_date, 'position_value'] = shares_held * next_row[stock]  # Use next day's price to update position value
        bt_data.loc[next_date, 'total_value'] = current_capital + bt_data.loc[next_date, 'position_value'] 

    bt_data.loc[bt_data.index[0],'position_value'] = 0
    bt_data.loc[bt_data.index[0],'total_value'] = initial_capital
    bt_data['return'] = bt_data['total_value'].pct_change()
    bt_data = bt_data.dropna()
    return bt_data

In [40]:
def calculate_performance_metrics(result:pd.DataFrame, rf, sortino_benchmark):
    stock = result.columns[0]

    total_return = (result['total_value'].iloc[-1]-result['total_value'].iloc[0])/result['total_value'].iloc[0]
    print(f"total return for {stock}: ", total_return)

    years = (result.index[-1] - result.index[0]).days / 252
    annual_return = (1 + total_return) ** (1/years) - 1
    print(f"annual return for {stock}: ", annual_return)

    annual_volatility = result['return'].std() * np.sqrt(252)  
    print(f"annual volatility for {stock}: ", annual_volatility)

    sharpe_ratio = (annual_return - rf) / annual_volatility
    print(f"sharpe ratio for {stock}: ", sharpe_ratio)

    downside_returns = result['return'][result['return']<sortino_benchmark]
    downside_volatility = downside_returns.std() * np.sqrt(252)
    sortino_ratio = (annual_return - rf) / downside_volatility
    print(f"sortino ratio under benchmark {sortino_benchmark} for {stock}: ", sortino_ratio)

    roll_max = result['total_value'].cummax()
    daily_drawdown =1 - result['total_value']/roll_max
    max_drawdown = daily_drawdown.max()
    print(f"max drawdown for {stock}: ", max_drawdown)

    print("----")



In [45]:
tickers = ['MSFT', 'AAPL', 'NVDA', 'AMZN', 'GOOG', 'META', 'TSLA']
start_date = '2013-01-01'
end_date = '2023-12-31'

# download data from yahoo finance and calculate variables for the double bollinger bands strategy
data = prepare_data(tickers, start_date, end_date,1, 1.5)
data.head()

[*********************100%%**********************]  7 of 7 completed


Ticker,AAPL,AMZN,GOOG,META,MSFT,NVDA,TSLA,MSFT_ub_1,MSFT_lb_1,MSFT_ub_2,...,GOOG_ub_2,GOOG_lb_2,META_ub_1,META_lb_1,META_ub_2,META_lb_2,TSLA_ub_1,TSLA_lb_1,TSLA_ub_2,TSLA_lb_2
Date,,,,,,,,,,,,,,,,,,,,,
2013-01-30,16.315357,13.6380,18.775375,31.240000,27.850000,3.0800,2.501333,27.701472,26.750527,27.939209,...,18.858777,17.659892,31.503848,29.095152,32.106023,28.492978,2.443372,2.232961,2.495975,2.180358
2013-01-31,16.267500,13.2750,18.821701,30.980000,27.450001,3.0650,2.500667,27.687339,26.747661,27.922259,...,18.919775,17.679691,31.538006,29.358994,32.082759,28.814241,2.456323,2.234343,2.511818,2.178848
2013-02-01,16.200714,13.2500,19.317593,29.730000,27.930000,3.0925,2.553333,27.746397,26.756603,27.993845,...,19.059425,17.669382,31.465474,29.627526,31.924961,29.168039,2.476708,2.237492,2.536512,2.177687
2013-02-04,15.797143,12.9990,18.904640,28.110001,27.440001,3.0400,2.516000,27.768571,26.804429,28.009607,...,19.107774,17.673461,31.504305,29.523695,31.999458,29.028542,2.491689,2.244777,2.553417,2.183049
2013-02-05,16.351429,13.3445,19.072014,28.639999,27.500000,3.1100,2.542000,27.790941,26.863059,28.022911,...,19.179061,17.679359,31.521392,29.428608,32.044587,28.905413,2.508461,2.253273,2.572258,2.189476


In [48]:
# choose the stock and initial capital to call the strategy function
stock = 'AAPL'
initial_capital =10000

# use 2016-2020 to backtest. change the part below for different backtest periods
result = Double_Bollinger_Bands(data, stock,'2016-01-01', '2020-01-01', initial_capital)
print(result['return'].value_counts())
result

return
 0.000000    86
 0.006781     1
-0.017568     1
-0.021369     1
-0.023356     1
             ..
 0.028534     1
-0.009066     1
-0.004019     1
 0.010062     1
-0.002067     1
Name: count, Length: 174, dtype: int64


Ticker,AAPL,AAPL_ub_1,AAPL_lb_1,AAPL_ub_2,AAPL_lb_2,Signal,position_value,total_value,return
Date,,,,,,,,,
2016-01-12,24.990000,27.516327,25.216673,28.091240,24.641760,SELL,0.000000,10000.000000,0.000000
2016-01-14,24.879999,27.179905,24.901594,27.749483,24.332016,SELL,0.000000,10000.000000,0.000000
2016-01-15,24.282499,26.985502,24.740748,27.546691,24.179559,SELL,0.000000,10000.000000,0.000000
2016-01-19,24.165001,26.842578,24.575671,27.409305,24.008945,SELL,0.000000,10000.000000,0.000000
2016-01-20,24.197500,26.757135,24.430114,27.338891,23.848359,SELL,0.000000,10000.000000,0.000000
...,...,...,...,...,...,...,...,...,...
2019-11-27,66.959999,66.533585,64.264417,67.100877,63.697124,BUY,18815.759743,18860.312277,0.013400
2019-11-29,66.812500,66.535044,64.725208,66.987503,64.272749,BUY,18774.312500,18818.865034,-0.002198
2019-12-03,64.862503,66.546866,64.970636,66.940924,64.576578,SELL,18226.363358,18270.915892,-0.017787


In [49]:
# calculate metrics: set risk_free rate = 0.02, downside benchmark = 0
calculate_performance_metrics(result, 0.02, 0)

total return for AAPL:  0.9391592947006225
annual return for AAPL:  0.12305845905232493
annual volatility for AAPL:  0.125632777558071
sharpe ratio for AAPL:  0.8203150567509216
sortino ratio under benchmark 0 for AAPL:  0.9566308209056268
max drawdown for AAPL:  0.20544020128939489
----
